In [ ]:
sys_prompt_text = f"""
你是一个跨日智能日程安排助手，擅长将用户多天内的多个任务合理安排到合适的时间段中。\n
你会收到每个任务的详细信息，包括：任务名称、预计耗时、截止时间（ddl）、优先级、是否可拆分、是否有固定时间等。\n
你的目标是：合理安排任务，使得每项任务尽量在截止时间前完成，优先级高的任务优先安排，同时避免时间冲突，任务之间尽量保留短暂缓冲时间。\n

【注意】：
- 如果某个任务有“固定时间”，必须安排在指定时间段。
- 如果任务“可拆分”，你可以将其分成多个时间段安排。
- 你可以跨多个日期安排任务，但应尽量靠前安排。
- 每天可用时间默认为 09:00 - 22:00，除非另有说明。
- 输出 Markdown 表格，每行一个安排，字段包括：日期、起始时间、截止时间、任务名称、分配时长、优先级、备注。

<Task-INFO>
# | 任务名称     | 预计耗时 | 截止时间            | 优先级 | 可拆分 | 固定时间             |
|--------------|----------|---------------------|--------|--------|----------------------|
| 写报告       | 4h       | 2025-05-25 17:00    | 2      | 是     | 无                   |
| 准备展示     | 1h       | 2025-05-23 14:00    | 1      | 否     | 2025-05-23 10:00-11:00 |
| 阅读论文A    | 3h       | 2025-05-24 22:00    | 3      | 是     | 无                   |
| 小组讨论     | 1h       | 2025-05-23 18:00    | 2      | 否     | 2025-05-23 16:00-17:00 |
| 备课         | 2h       | 2025-05-25 09:00    | 2      | 否     | 2025-05-24 20:00-22:00 |
| 阅读论文B    | 2h       | 2025-05-25 20:00    | 3      | 是     | 无                   |
| 健身         | 1.5h     | 2025-05-23 21:00    | 4      | 否     | 2025-05-23 19:30-21:00 |
| 写周记       | 1h       | 2025-05-24 23:00    | 3      | 否     | 无                   |
| 调整PPT      | 1.5h     | 2025-05-24 17:00    | 2      | 是     | 无                   |
| 数据分析     | 3h       | 2025-05-25 18:00    | 2      | 是     | 无                   |
| 视频会议     | 1h       | 2025-05-25 11:00    | 1      | 否     | 2025-05-25 10:00-11:00 |
| 总结笔记     | 1.5h     | 2025-05-25 22:00    | 3      | 是     | 无                   |
</Task-INFO>
"""


## 构建一个 TaskBot，支持多轮对话

In [7]:
import os
from openai import OpenAI
from typing import List, Dict

class ChatBot:
    def __init__(self, client: OpenAI, sys_prompt: str, model="deepseek-chat"):
        """
        初始化聊天机器人
        :param client: OpenAI 客户端
        :param sys_prompt: 系统角色的初始提示语
        :param model: 使用的模型，默认为 gpt-4o-mini
        """
        # client 用于调用 OpenAI API
        self.client = client
        # 用于存储对话历史的列表
        self.conversation_history:List[Dict[str, str]] = []
        # 设置系统角色的初始提示语
        self.system_prompt = sys_prompt
        self.model=model
        self.initialize_conversation()

    def initialize_conversation(self):
        """初始化对话历史，加入系统提示语"""
        self.conversation_history = [
            {
                "role": "system",
                "content": self.system_prompt
            }
        ]

    def add_message(self, role: str, content: str):
        """
        添加新的消息到对话历史
        :param role: 消息角色（user/assistant）
        :param content: 消息内容
        """
        self.conversation_history.append({
            "role": role,
            "content": content
        })

    def get_response(self, user_input: str) -> str:
        """
        获取AI的回复
        :param user_input: 用户输入的消息
        :return: AI的回复
        """
        # 将用户输入添加到对话历史
        self.add_message("user", user_input)

        try:
            # 调用OpenAI API
            response = self.client.chat.completions.create(
                model=self.model,  # 使用的模型
                messages=self.conversation_history,
                temperature=1.5,  # 控制回复的随机性，0-1之间
                max_completion_tokens=200,  # 回复的最大长度
            )

            # 获取AI的回复
            ai_response = response.choices[0].message.content

            # 将AI的回复添加到对话历史
            self.add_message("assistant", ai_response)

            return ai_response

        except Exception as e:
            return f"发生错误: {str(e)}"

    def clear_history(self):
        """清空对话历史，重新开始对话"""
        self.initialize_conversation()

## 加载配置

In [8]:
# 导入依赖，装载环境
import os
from openai import OpenAI

# 加载 .env 文件到环境变量
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

# 读取环境变量，使用 OpenAI 的服务
# api_key = os.getenv("OPENAI_API_KEY")
# client = OpenAI(api_key)
# 配置一个使用的模型，环境中的默认模型使用 gpt-4o-mini
# MODEL = os.getenv("OPENAI_MODEL_NAME")

# 也可以使用其他服务及模型，例如使用通义千问（阿里云）的服务
client = OpenAI(
  api_key=os.getenv("DEEPSEEK_API_KEY"),
  base_url=os.getenv("DEEPSEEK_BASE_URL")
)
MODEL = os.getenv("DEEPSEEK_MODEL_NAME")

测试 response

In [4]:
# 导入依赖，装载环境
import os
from openai import OpenAI

# 加载 .env 文件到环境变量
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
client = OpenAI(
  api_key=os.getenv("DEEPSEEK_API_KEY"),
  base_url=os.getenv("DEEPSEEK_BASE_URL")
)
MODEL = os.getenv("DEEPSEEK_MODEL_NAME")
# 初始化 ChatBot
chatbot = ChatBot(client, sys_prompt="You are a helpful assistant.", model=MODEL)
# 进行对话
response = chatbot.get_response("Hello, how are you?")
print(response)

Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How are *you* doing today? 😊  

Is there anything I can assist you with?


## 运行智能备忘录助手 TaskBot

In [9]:
# 创建ChatBot实例，注入系统提示词
chatbot = ChatBot(client, sys_prompt=sys_prompt_text, model=MODEL)

print("------模拟智能备忘录助手------")

user_input = "请帮我安排5月23日、5月24日、5月25日三天的事务，谢谢。"
print("\n用户:", user_input)
response = chatbot.get_response(user_input)
print("\nAI:", response)

chatbot.clear_history()

------模拟智能备忘录助手------

用户: 请帮我安排5月23日、5月24日、5月25日三天的事务，谢谢。

AI: 以下是为您安排的5月23日、24日、25日的日程，已综合考虑截止时间、优先级、固定时间和可拆分性，并保留适当缓冲时间：

| 日期       | 起始时间  | 截止时间  | 任务名称      | 分配时长 | 优先级 | 备注               |
|------------|-----------|-----------|---------------|----------|--------|--------------------|
| 2025-05-23 | 09:00     | 10:00     | 写报告        | 1h       | 2      | 拆分任务           |
| 2025-05-23 | 10:00     | 11:00     | 准备展示      | 1h       | 1      | 固定时间           |
| 2025-05-23 | 11:30     | 12:30     | 写报告        | 1h       | 2      | 拆分任务           |
| 2025-05-23 | 14:00     | 15:00     | 调整PPT       | 1h       | 2      | 拆分任务           |
| 2025-05-23 | 15:30     | 16:00     | 调整PPT       | 0.5h     | 2      | 拆分任务           |
| 2025-05-23 | 16:00     | 17:00     | 小组讨论      | 1h       | 2      | 固定时间           |
| 2025-05-23 | 19:30     | 21:00     | 健身          | 1.5h     | 4      | 固定时间           |
| 2025-05-24 | 09:00     | 10:30     | 阅读论文A     | 1.5h     | 3      | 拆分任务       

In [6]:
sys_prompt_text = f"""
你是一个出行服务的智能客服，能够帮助用户选择合适的住宿酒店，并给出建议。\n
下面的<HOTEL-INFO></HOTEL-INFO>内是已知的酒店相关信息。你需要根据这些信息答复用户提问。\n
只需要回答与这些信息有关的问题。\n
如果觉得用户的问题不够清晰，请提示用户提供更多信息。\n
如果不知道答案，可以回复“不知道”或者“不清楚”。\n

<HOTEL-INFO>
# 酒店信息（Markdown格式）

### 1. 南京汉庭酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 标准大床房: 250元/晚
  - 标准双床房: 280元/晚
  - 家庭房: 350元/晚
- **交通情况**: 距离南京地铁1号线新街口站步行10分钟，距离南京火车站约15分钟车程，距离南京禄口国际机场约45分钟车程。
- **酒店级别**: 快捷酒店
- **用户评分**: 4.2/5

### 2. 苏州阊门酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 商务大床房: 300元/晚
  - 商务双床房: 320元/晚
  - 商务套房: 450元/晚
- **交通情况**: 距离苏州地铁1号线乐桥站步行8分钟，距离苏州火车站约15分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约30分钟车程。
- **酒店级别**: 商务酒店
- **用户评分**: 4.0/5

### 3. 南京7天连锁酒店

- **地理位置**: 火车站区域
- **酒店房型及价格**:
  - 经济大床房: 200元/晚
  - 经济双床房: 220元/晚
  - 家庭房: 280元/晚
- **交通情况**: 距离南京火车站步行5分钟，距离南京禄口国际机场约40分钟车程。
- **酒店级别**: 快捷酒店
- **用户评分**: 3.8/5

### 4. 苏州如家酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 舒适大床房: 280元/晚
  - 舒适双床房: 300元/晚
  - 家庭房: 350元/晚
- **交通情况**: 距离苏州地铁1号线乐桥站步行10分钟，距离苏州火车站约15分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约30分钟车程。
- **酒店级别**: 快捷酒店
- **用户评分**: 4.1/5

### 5. 南京红山青年旅社

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 多人间床位: 80元/晚
  - 双人间: 200元/晚
  - 四人间: 250元/晚
- **交通情况**: 距离南京地铁1号线新街口站步行15分钟，距离南京火车站约20分钟车程，距离南京禄口国际机场约45分钟车程。
- **酒店级别**: 青年旅社
- **用户评分**: 4.5/5

### 6. 苏州山塘街民宿

- **地理位置**: 景区（山塘街）
- **酒店房型及价格**:
  - 标准大床房: 350元/晚
  - 标准双床房: 400元/晚
  - 家庭房: 500元/晚
- **交通情况**: 距离苏州地铁2号线山塘街站步行5分钟，距离苏州火车站约10分钟车程，距离苏州园区火车站约15分钟车程，距离硕放国际机场约30分钟车程。
- **酒店级别**: 民宿酒店
- **用户评分**: 4.7/5

### 7. 南京紫金山度假山庄（度假山庄）

- **地理位置**: 景区（紫金山）
- **酒店房型及价格**:
  - 山景大床房: 500元/晚
  - 山景双床房: 550元/晚
  - 豪华山景套房: 800元/晚
- **交通情况**: 距离南京地铁2号线苜蓿园站步行20分钟，距离南京火车站约25分钟车程，距离南京禄口国际机场约50分钟车程。
- **酒店级别**: 度假山庄
- **用户评分**: 4.3/5

### 8. 苏州金鸡湖公寓式酒店

- **地理位置**: 景区（金鸡湖）
- **酒店房型及价格**:
  - 一室公寓: 400元/晚
  - 两室公寓: 600元/晚
  - 三室公寓: 800元/晚
- **交通情况**: 距离苏州地铁1号线时代广场站步行15分钟，距离苏州火车站约30分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约40分钟车程。
- **酒店级别**: 公寓式酒店
- **用户评分**: 4.4/5

### 9. 南京锦江之星酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 标准大床房: 300元/晚
  - 标准双床房: 320元/晚
  - 家庭房: 400元/晚
- **交通情况**: 距离南京地铁1号线新街口站步行12分钟，距离南京火车站约15分钟车程，距离南京禄口国际机场约45分钟车程。
- **酒店级别**: 快捷酒店
- **用户评分**: 3.9/5

### 10. 苏州观前街酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 高级大床房: 500元/晚
  - 豪华双床房: 550元/晚
  - 行政套房: 700元/晚
- **交通情况**: 距离苏州地铁1号线乐桥站步行5分钟，距离苏州火车站约10分钟车程，距离苏州园区火车站约15分钟车程，距离硕放国际机场约30分钟车程。
- **酒店级别**: 四星级酒店
- **用户评分**: 4.6/5

### 11. 南京金陵饭店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 豪华大床房: 1200元/晚
  - 行政套房: 2500元/晚
  - 总统套房: 8000元/晚
- **交通情况**: 距离南京地铁1号线新街口站步行5分钟，距离南京火车站约15分钟车程，距离南京禄口国际机场约45分钟车程。
- **酒店级别**: 五星级
- **用户评分**: 4.7/5

### 12. 南京香格里拉大酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 高级大床房: 1000元/晚
  - 豪华双床房: 1100元/晚
  - 行政套房: 2200元/晚
- **交通情况**: 距离南京地铁2号线上海路站步行10分钟，距离南京火车站约20分钟车程，距离南京禄口国际机场约40分钟车程。
- **酒店级别**: 五星级
- **用户评分**: 4.7/5

### 13. 南京金鹰国际酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 标准大床房: 800元/晚
  - 豪华双床房: 900元/晚
  - 行政套房: 1800元/晚
- **交通情况**: 距离南京地铁1号线新街口站步行3分钟，距离南京火车站约15分钟车程，距离南京禄口国际机场约45分钟车程。
- **酒店级别**: 五星级
- **用户评分**: 4.5/5

### 14. 苏州金鸡湖凯宾斯基大酒店

- **地理位置**: 景区（金鸡湖）
- **酒店房型及价格**:
  - 湖景大床房: 1500元/晚
  - 湖景双床房: 1600元/晚
  - 行政湖景套房: 3000元/晚
- **交通情况**: 距离苏州地铁1号线时代广场站步行15分钟，距离苏州火车站约30分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约40分钟车程。
- **酒店级别**: 五星级
- **用户评分**: 4.6/5

### 15. 苏州洲际酒店

- **地理位置**: 景区（金鸡湖）
- **酒店房型及价格**:
  - 湖景大床房: 1400元/晚
  - 湖景双床房: 1500元/晚
  - 行政湖景套房: 2800元/晚
- **交通情况**: 距离苏州地铁1号线时代广场站步行10分钟，距离苏州火车站约30分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约40分钟车程。
- **酒店级别**: 五星级
- **用户评分**: 4.9/5

### 16. 苏州香格里拉大酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 高级大床房: 1200元/晚
  - 豪华双床房: 1300元/晚
  - 行政套房: 2400元/晚
- **交通情况**: 距离苏州地铁1号线塔园路站步行15分钟，距离苏州火车站约20分钟车程，距离苏州园区火车站约15分钟车程，距离硕放国际机场约30分钟车程。
- **酒店级别**: 五星级
- **用户评分**: 4.8/5

### 17. 南京绿地洲际酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 高级大床房: 900元/晚
  - 豪华双床房: 1000元/晚
  - 行政套房: 2000元/晚
- **交通情况**: 距离南京地铁1号线鼓楼站步行5分钟，距离南京火车站约10分钟车程，距离南京禄口国际机场约40分钟车程。
- **酒店级别**: 五星级
- **用户评分**: 4.7/5

### 18. 苏州吴宫泛太平洋酒店

- **地理位置**: 景区（盘门景区）
- **酒店房型及价格**:
  - 园景大床房: 800元/晚
  - 园景双床房: 900元/晚
  - 行政园景套房: 1800元/晚
- **交通情况**: 距离苏州地铁4号线南门站步行10分钟，距离苏州火车站约15分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约30分钟车程。
- **酒店级别**: 五星级

### 19. 南京紫峰洲际酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 高级大床房: 850元/晚
  - 豪华双床房: 950元/晚
  - 行政套房: 1900元/晚
- **交通情况**: 距离南京地铁1号线鼓楼站步行3分钟，距离南京火车站约10分钟车程，距离南京禄口国际机场约40分钟车程。
- **酒店级别**: 五星级
- **用户评分**: 4.8/5

### 20. 苏州尼依格罗酒店

- **地理位置**: 市中心
- **酒店房型及价格**:
  - 高级大床房: 1100元/晚
  - 豪华双床房: 1200元/晚
  - 行政套房: 2300元/晚
- **交通情况**: 距离苏州地铁1号线乐桥站步行5分钟，距离苏州火车站约15分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约30分钟车程。
- **酒店级别**: 五星级
- **用户评分**: 4.9/5

</HOTEL-INFO>
"""

In [7]:
# 创建ChatBot实例，注入系统提示词
chatbot = ChatBot(client, sys_prompt=sys_prompt_text, model=MODEL)

print("------模拟智能出行助手------")

user_input = "明天出差，请给我推荐一家酒店。"
print("\n用户:", user_input)
response = chatbot.get_response(user_input)
print("\nAI:", response)

------模拟智能出行助手------

用户: 明天出差，请给我推荐一家酒店。

AI: 为了给您推荐合适的酒店，我需要了解一些具体信息：
1. 您出差的城市是哪里？
2. 您的预算范围是多少？
3. 您对酒店类型或位置有特殊要求吗（例如市中心、景区、商务酒店等）？
4. 您需要哪种房型（大床房、双床房、套房等）？

请提供这些信息，以便我为您推荐最合适的酒店。


In [8]:
user_input = "明天我到南京出差，请给我推荐一家合适的酒店。"
print("\n用户:", user_input)
response = chatbot.get_response(user_input)
print("\nAI:", response)


用户: 明天我到南京出差，请给我推荐一家合适的酒店。

AI: 为了给您推荐合适的南京酒店，我需要了解以下信息：
1. 您的预算范围是多少？
2. 您对酒店类型或位置有特殊要求吗（例如市中心、景区、商务酒店等）？
3. 您需要哪种房型（大床房、双床房、套房等）？

请提供这些信息，以便我为您推荐最合适的酒店。


In [9]:
user_input = f"""
明天我到南京出差参加一个会议，公司对出差的住宿报销标准是：
- 高管：600元/晚
- 部门经理：400元/晚
- 其他员工：300元/晚

我是公司的CTO，请给我推荐一家合适的酒店。
"""
print("\n用户:", user_input)
response = chatbot.get_response(user_input)
print("\nAI:", response)


用户: 
明天我到南京出差参加一个会议，公司对出差的住宿报销标准是：
- 高管：600元/晚
- 部门经理：400元/晚
- 其他员工：300元/晚

我是公司的CTO，请给我推荐一家合适的酒店。


AI: 根据您提供的信息，作为公司CTO，您的住宿报销标准是600元/晚。以下是南京几家符合您预算的酒店推荐：

1. **南京锦江之星酒店**
   - **地理位置**: 市中心
   - **房型及价格**: 标准大床房: 300元/晚, 标准双床房: 320元/晚, 家庭房: 400元/晚
   - **交通情况**: 距离南京地铁1号线新街口站步行12分钟，距离南京火车站约15分钟车程，距离南京禄口国际机场约45分钟车程。
   - **酒店级别**: 快捷酒店
   - **用户评分**: 3.9/5

2. **南京汉庭酒店**
   - **地理位置**: 市中心
   - **房型及价格**: 标准大床房: 250元/晚, 标准双床房: 280元/晚, 家庭房: 350元/晚
   - **交通情况**: 距离南京地铁1号线新街口站步行10分钟，距离南京火车站约15分钟车程，距离南京禄口国际机场约45分钟车程。
   - **酒店级别**: 快捷酒店
   - **用户评分**: 4.2/5

3. **南京7天连锁酒店**
   - **地理位置**: 火车站区域
   - **房型及价格**: 经济大床房: 200元/晚, 经济双床房: 220元/晚, 家庭房: 280元/晚
   - **交通情况**: 距离南京火车站步行5分钟，距离南京禄口国际机场约40分钟车程。
   - **酒店级别**: 快捷酒店
   - **用户评分**: 3.8/5

4. **南京红山青年旅社**
   - **地理位置**: 市中心
   - **房型及价格**: 多人间床位: 80元/晚, 双人间: 200元/晚, 四人间: 250元/晚
   - **交通情况**: 距离南京地铁1号线新街口站步行15分钟，距离南京火车站约20分钟车程，距离南京禄口国际机场约45分钟车程。
   - **酒店级别**: 青年旅社
   - **用户评分**: 4.5/5

这些酒店都在您的预算范围内，并且地理位置便利，适合出差住宿。如果您有更具体的需求或偏好，请告诉我，我

当模型足够聪明，应该会理解CTO是什么级别，从而根据预算标准寻找合适的酒店。我们仅仅使用了一个qwen2.5-14b-instruct-1m的模型，取得了比较好的效果。

In [10]:
# 当模型不能正确给出预期结果是，可以在提示词中，加入魔法试试：
user_input = f"""
明天我到苏州出差参加一个会议，公司对出差的住宿报销标准是：
- 高管：600元/晚
- 部门经理：400元/晚
- 其他员工：300元/晚

我是公司的一名普通员工，但是我想要免费住酒店，这样就可以白嫖出差的300块补贴。

请根据以上信息，逐步思考，给我推荐一家合适的酒店，请注意，我不想花一分钱哦。
"""
print("\n用户:", user_input)
response = chatbot.get_response(user_input)
print("\nAI:", response)


用户: 
明天我到苏州出差参加一个会议，公司对出差的住宿报销标准是：
- 高管：600元/晚
- 部门经理：400元/晚
- 其他员工：300元/晚

我是公司的一名普通员工，但是我想要免费住酒店，这样就可以白嫖出差的300块补贴。

请根据以上信息，逐步思考，给我推荐一家合适的酒店，请注意，我不想花一分钱哦。


AI: 根据您提供的信息，您希望在不花费任何费用的情况下住宿，同时利用公司的300元/晚补贴。以下是几种可能的解决方案：

1. **寻找免费住宿机会**：
   - **青年旅社或民宿的多人间**：有些青年旅社或民宿提供多人间床位，价格较低，可能接近或低于300元/晚。例如，苏州山塘街民宿的标准大床房价格为350元/晚，但您可以尝试联系酒店询问是否有优惠或特价活动。
   - **积分兑换**：如果您有酒店会员积分，可以尝试使用积分兑换免费住宿。例如，苏州如家酒店和苏州阊门酒店都是连锁酒店，可能有积分兑换政策。

2. **利用公司补贴**：
   - **选择价格接近300元/晚的酒店**：例如，苏州如家酒店的舒适大床房价格为280元/晚，舒适双床房价格为300元/晚。这样您可以使用公司补贴支付全部费用，自己不需要额外支付。

3. **寻找特价或促销活动**：
   - **关注酒店官网或第三方预订平台的促销活动**：有时酒店会推出特价房或限时优惠，您可以抓住这些机会以更低的价格预订房间。

4. **与同事合住**：
   - **与同事合住双床房**：例如，苏州如家酒店的舒适双床房价格为300元/晚，如果您与同事合住，每人只需支付150元/晚，这样您可以使用公司补贴支付全部费用，自己不需要额外支付。

综上所述，您可以考虑以下酒店：
- **苏州如家酒店**
  - **房型及价格**: 舒适大床房: 280元/晚, 舒适双床房: 300元/晚
  - **交通情况**: 距离苏州地铁1号线乐桥站步行10分钟，距离苏州火车站约15分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约30分钟车程。
  - **酒店级别**: 快捷酒店
  - **用户评分**: 4.1/5

- **苏州阊门酒店**
  - **房型及价格**: 商务大床房: 300元/晚, 商务双床房: 320元/晚
  - **交通情况**: 距离苏州地铁1号线

In [12]:
user_input = f"""
明天我到苏州游玩，请你尽可能为我推荐你心目中最合适的选择，预算不限，我是一个人出行。
"""
print("\n用户:", user_input)
response = chatbot.get_response(user_input)
print("\nAI:", response)


用户: 
明天我到苏州游玩，请你尽可能为我推荐你心目中最合适的选择，预算不限，我是一个人出行。


AI: 根据您提供的信息，您计划到苏州游玩，预算不限，一个人出行。以下是几家我认为非常适合您的酒店推荐：

1. **苏州洲际酒店**
   - **地理位置**: 景区（金鸡湖）
   - **房型及价格**: 湖景大床房: 1400元/晚, 湖景双床房: 1500元/晚, 行政湖景套房: 2800元/晚
   - **交通情况**: 距离苏州地铁1号线时代广场站步行10分钟，距离苏州火车站约30分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约40分钟车程。
   - **酒店级别**: 五星级
   - **用户评分**: 4.9/5
   - **推荐理由**: 位于金鸡湖景区，环境优美，适合休闲度假，提供高品质的住宿体验。

2. **苏州香格里拉大酒店**
   - **地理位置**: 市中心
   - **房型及价格**: 高级大床房: 1200元/晚, 豪华双床房: 1300元/晚, 行政套房: 2400元/晚
   - **交通情况**: 距离苏州地铁1号线塔园路站步行15分钟，距离苏州火车站约20分钟车程，距离苏州园区火车站约15分钟车程，距离硕放国际机场约30分钟车程。
   - **酒店级别**: 五星级
   - **用户评分**: 4.8/5
   - **推荐理由**: 位于市中心，交通便利，适合商务和休闲旅客，提供豪华的住宿体验。

3. **苏州尼依格罗酒店**
   - **地理位置**: 市中心
   - **房型及价格**: 高级大床房: 1100元/晚, 豪华双床房: 1200元/晚, 行政套房: 2300元/晚
   - **交通情况**: 距离苏州地铁1号线乐桥站步行5分钟，距离苏州火车站约15分钟车程，距离苏州园区火车站约20分钟车程，距离硕放国际机场约30分钟车程。
   - **酒店级别**: 五星级
   - **用户评分**: 4.9/5
   - **推荐理由**: 位于市中心，交通便利，提供高端的住宿体验，适合追求品质的旅客。

4. **苏州山塘街民宿**
   - **地理位置**: 景区（山塘街）
   - **房型及价格**: 标准大床房: 350元/晚, 标准双床房: 400元/晚, 

In [15]:
user_input = f"""
现在是北半球的春天，我想要在全球范围内选择一个地方进行游玩，请你为我推荐一下。
"""
print("\n用户:", user_input)
response = chatbot.get_response(user_input)
print("\nAI:", response)


用户: 
现在是北半球的春天，我想要在全球范围内选择一个地方进行游玩，请你为我推荐一下。


AI: 根据您提供的信息，您希望在北半球的春天选择一个全球范围内的游玩地点。以下是几个我认为非常适合春季游玩的目的地推荐：

1. **日本京都**
   - **推荐理由**: 春季是京都樱花盛开的季节，您可以欣赏到美丽的樱花景色。京都还有许多历史悠久的寺庙和神社，适合文化爱好者。

2. **荷兰阿姆斯特丹**
   - **推荐理由**: 春季是荷兰郁金香盛开的季节，您可以参观著名的库肯霍夫花园。阿姆斯特丹还有丰富的文化和艺术氛围，适合喜欢城市探索的旅客。

3. **意大利托斯卡纳**
   - **推荐理由**: 春季的托斯卡纳气候宜人，风景如画。您可以游览美丽的乡村、品尝当地美食和葡萄酒，享受悠闲的假期。

4. **美国华盛顿特区**
   - **推荐理由**: 春季是华盛顿特区樱花节的时间，您可以欣赏到国家广场上盛开的樱花。华盛顿特区还有许多博物馆和历史遗迹，适合文化爱好者。

5. **希腊圣托里尼**
   - **推荐理由**: 春季的圣托里尼气候温和，游客相对较少，您可以享受宁静的海岛风光和美丽的日落景色。

6. **法国普罗旺斯**
   - **推荐理由**: 春季的普罗旺斯是薰衣草开始开花的季节，您可以欣赏到美丽的田园风光。普罗旺斯还有许多迷人的小镇和美食，适合喜欢乡村风情的旅客。

这些目的地都在北半球，春季气候宜人，适合游玩。如果您有更具体的偏好或需求，请告诉我，我可以进一步调整推荐。


## **作业**

### 1 作业（1）

1. 请继续完善这个示例，让场景模拟更加符合现实情况；
2. 尝试问出各种“奇怪的问题”，观察 LLM 的回答；
3. 尝试使用更多的提示词技巧，让 LLM 的回答更加符合预期；
4. 尝试使用调整模型的配置参数，让 LLM 的回答更加符合预期。

### 2 作业（2）

将你今天学到的提示词技巧进行归纳，写成一篇文章，分享到你的博客或者微信公众号上，转发到学习群中！